In [1]:
from PIL import Image, ImageFilter

kitten = Image.open('../data/kitten.jpg')
blurryKitten = kitten.filter(ImageFilter.GaussianBlur)
blurryKitten.save('../data/kitten_blurred.jpg')
blurryKitten.show()

In [2]:
from PIL import Image
import pytesseract

print(pytesseract.image_to_string(Image.open('../data/logo.jpg')))

In [3]:
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

In [4]:
print(pytesseract.image_to_boxes(Image.open('../data/logo.jpg')))

~ 0 0 163 160 0



In [5]:
print(pytesseract.image_to_data(Image.open('../data/logo.jpg')))

level	page_num	block_num	par_num	line_num	word_num	left	top	width	height	conf	text
1	1	0	0	0	0	0	0	163	160	-1	
2	1	1	0	0	0	0	0	163	160	-1	
3	1	1	1	0	0	0	0	163	160	-1	
4	1	1	1	1	0	0	0	163	160	-1	
5	1	1	1	1	1	0	0	163	160	95	 



In [6]:
from PIL import Image
import pytesseract
from pytesseract import Output

print(pytesseract.image_to_data(Image.open('../data/logo.jpg'),
   output_type=Output.DICT))
print(pytesseract.image_to_string(Image.open('../data/logo.jpg'),
   output_type=Output.BYTES))

{'level': [1, 2, 3, 4, 5], 'page_num': [1, 1, 1, 1, 1], 'block_num': [0, 1, 1, 1, 1], 'par_num': [0, 0, 1, 1, 1], 'line_num': [0, 0, 0, 1, 1], 'word_num': [0, 0, 0, 0, 1], 'left': [0, 0, 0, 0, 0], 'top': [0, 0, 0, 0, 0], 'width': [163, 163, 163, 163, 163], 'height': [160, 160, 160, 160, 160], 'conf': ['-1', '-1', '-1', '-1', 95], 'text': ['', '', '', '', '']}
b' \n\x0c'


In [7]:
import numpy as np

numbers = [100, 102, 98, 97, 103]
print(np.std(numbers))
print(np.mean(numbers))

2.280350850198276
100.0


In [8]:
from PIL import Image
import pytesseract

def cleanFile(filePath, newFilePath):
    image = Image.open(filePath)

    #Set a threshold value for the image, and save
    image = image.point(lambda x: 0 if x < 143 else 255)
    image.save(newFilePath)
    return image

image = cleanFile('../data/textBad.png', '../data/textCleaned.png')

#call tesseract to do OCR on the newly created image
print(pytesseract.image_to_string(image))

This 1s some text, written in Arial, that will be read by
Tesseract Here are some symbols: |@#$%"&*(}



In [27]:
import pytesseract
from pytesseract import Output
from PIL import Image
import numpy as np

def cleanFile(filePath, threshold):
    image = Image.open(filePath)
    #Set a threshold value for the image, and save
    image = image.point(lambda x: 0 if x<threshold else 255)
    return image

def getConfidence(image):
    data = pytesseract.image_to_data(image, output_type=Output.DICT)
    text = data['text']
    confidences = []
    numChars = []

    for i in range(len(text)):
        if int(data['conf'][i]) > -1:
            confidences.append(data['conf'][i])
            numChars.append(len(text[i]))

    return np.average(confidences, weights=numChars), sum(numChars)

filePath = '../data/textBad.png'

start = 80
step = 5
end = 200

for threshold in range(start, end, step):
    image = cleanFile(filePath, threshold)
    scores = getConfidence(image)
    print('threshold: {}, confidence: {}, numChars {}'.format(str(threshold), scores[0], str(scores[1])))

threshold: 80, confidence: 35.61538461538461, numChars 13
threshold: 85, confidence: 35.31818181818182, numChars 22
threshold: 90, confidence: 31.525, numChars 40
threshold: 95, confidence: 26.26829268292683, numChars 41
threshold: 100, confidence: 38.916666666666664, numChars 60
threshold: 105, confidence: 49.378787878787875, numChars 66
threshold: 110, confidence: 57.12658227848101, numChars 79
threshold: 115, confidence: 67.32142857142857, numChars 84
threshold: 120, confidence: 69.8780487804878, numChars 82
threshold: 125, confidence: 60.06024096385542, numChars 83
threshold: 130, confidence: 73.78313253012048, numChars 83
threshold: 135, confidence: 76.04819277108433, numChars 83
threshold: 140, confidence: 81.46428571428571, numChars 84
threshold: 145, confidence: 79.91666666666667, numChars 84
threshold: 150, confidence: 72.9047619047619, numChars 84
threshold: 155, confidence: 77.48101265822785, numChars 79
threshold: 160, confidence: 82.47435897435898, numChars 78
threshold: 1

In [10]:
import time
from urllib.request import urlretrieve
from PIL import Image
import pytesseract
from selenium import webdriver
from PIL import Image

# Create new Selenium driver
driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver')

driver.get(
    'https://www.amazon.com/Death-Ivan-Ilyich-Nikolayevich-Tolstoy/dp/1427027277')
time.sleep(2)

# Click on the book preview button
driver.find_element_by_id('imgBlkFront').click()
imageList = []

# Wait for the page to load
time.sleep(5)

while 'pointer' in driver.find_element_by_id('sitbReaderRightPageTurner').get_attribute('style'):
    # While the right arrow is available for clicking, turn through pages
    driver.find_element_by_id('sitbReaderRightPageTurner').click()
    time.sleep(2)
    # Get any new pages that have loaded (multiple pages can load at once,
    # but duplicates will not be added to a set)
    pages = driver.find_elements_by_xpath(
        '//div[@class=\'pageImage\']/div/img')
    if not len(pages):
        print('No pages found')
    for page in pages:
        image = page.get_attribute('src')
        print('Found image: {}'.format(image))
        if image not in imageList:
            urlretrieve(image, 'page.jpg')
            imageList.append(image)
            print(pytesseract.image_to_string(Image.open('page.jpg')))

driver.quit()

Found image: https://d1b14unh5d6w7g.cloudfront.net/1427027277.01.S003.LXXXXXXX.jpg?Expires=1602301942&Signature=F6Y1+BuJUNWuA1j6/d12v4ffovOtIyLcO0WqoZgQalFlieBLrIEtKBCqWFCH1rExFn1ewyOZZxkFcQ0/+WhL1Q2uVnJZT1zGU/KQ06BpF9AVxNVuuv+BK9+GCm1WiVY1ME4iiOBwGFe4KJ6kC6vUAcnKici4ynuHawk9wUulde4=&Key-Pair-Id=APKAIUO27P366FGALUMQ
Copyrighted Material

Copyright © 2007

The text in this book has been formatted
and typeset to make reading easier and
more enjoyable. Spelling and punctuation
have been modernized and standardized,
and hyphens have been avoided when
possible. Obvious typographical errors
have been corrected. In addition, the text
has been formatted to the specifications
indicated on the title page. The original
text is out of copyright. The edited text in
this edition and the formatting are the
copyright of Objective Systems Pty Ltd
ACN 085 119 953.
. @

Copyrighted Material

Found image: https://d1b14unh5d6w7g.cloudfront.net/1427027277.01.S006.LXXXXXXX.jpg?Expires=1602301942&Signature=R

In [11]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import subprocess
import requests
from PIL import Image
from PIL import ImageOps

def cleanImage(imagePath):
    image = Image.open(imagePath)
    image = image.point(lambda x: 0 if x<143 else 255)
    borderImage = ImageOps.expand(image,border=20,fill='white')
    borderImage.save(imagePath)

html = urlopen('http://www.pythonscraping.com/humans-only')
bs = BeautifulSoup(html, 'html.parser')
#Gather prepopulated form values
imageLocation = bs.find('img', {'title': 'Image CAPTCHA'})['src']
formBuildId = bs.find('input', {'name':'form_build_id'})['value']
captchaSid = bs.find('input', {'name':'captcha_sid'})['value']
captchaToken = bs.find('input', {'name':'captcha_token'})['value']

captchaUrl = 'http://pythonscraping.com'+imageLocation
urlretrieve(captchaUrl, 'captcha.jpg')
cleanImage('captcha.jpg')
p = subprocess.Popen(['tesseract', 'captcha.jpg', 'captcha'], stdout=
    subprocess.PIPE,stderr=subprocess.PIPE)
p.wait()
f = open('captcha.txt', 'r')

#Clean any whitespace characters
captchaResponse = f.read().replace(' ', '').replace('\n', '')
print('Captcha solution attempt: '+captchaResponse)

if len(captchaResponse) == 5:
    params = {'captcha_token':captchaToken, 'captcha_sid':captchaSid,
              'form_id':'comment_node_page_form', 'form_build_id': formBuildId,
              'captcha_response':captchaResponse, 'name':'Ryan Mitchell',
              'subject': 'I come to seek the Grail',
              'comment_body[und][0][value]':
               '...and I am definitely not a bot'}
    r = requests.post('http://www.pythonscraping.com/comment/reply/10',
                          data=params)
    responseObj = BeautifulSoup(r.text, 'html.parser')
    if responseObj.find('div', {'class':'messages'}) is not None:
        print(responseObj.find('div', {'class':'messages'}).get_text())
else:
    print('There was a problem reading the CAPTCHA correctly!')

Captcha solution attempt: a3
There was a problem reading the CAPTCHA correctly!
